In [7]:
import os
import cv2 as cv
import matplotlib.pyplot as plt

In [8]:
# Load Folders in data_train
#basis ng pics na gin use for training
data_train_dir = [dir for dir in os.listdir("./data_train/")]

In [9]:
def preprocessing(img):
    # convert to grayscale
    preprocessed_img = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    # apply gaussian blur
    preprocessed_img = cv.GaussianBlur(preprocessed_img, (5,5), 0)
    # apply Canny Edge
    preprocessed_img = cv.Canny(image=preprocessed_img, threshold1=50, threshold2=150)
    return preprocessed_img

In [10]:
helmetLabels = [
    "DHelmetP1Helmet",
    "DHelmet",
    "DHelmetP1HelmetP2Helmet",
    "DHelmetP0Helmet",
    "DHelmetP0HelmetP1HelmetP2Helmet",
    "DHelmetP0HelmetP1Helmet"
]

In [11]:
os.makedirs("./Helmet/", exist_ok=True)
os.makedirs("./NoHelmet/", exist_ok=True)

In [12]:
data_counter = 0
#video_dir = Ex. Bago_highway_1.csv
#frame = images
for video_dir in range(len(data_train_dir)):
    for frame in range(100):
    # Loading sample image
        img_dir = [i for i in sorted(os.listdir(f"./data_train/{data_train_dir[video_dir]}"))]
        img_path = f"./data_train/{data_train_dir[video_dir]}/{img_dir[frame]}"
        img = cv.imread(img_path)
        

        # Create a bounding boxes based on annotations file
        boxes_lists = []
        label_texts = []
        annotation_file = f"./annotations/{data_train_dir[video_dir]}.csv"

        with open (annotation_file, "r") as x:
            for lines in x.readlines():
                temp = lines.split(",")
                try:
                    if int(temp[1]) == frame:
                        #based sa mga csv files
                        box = (int(temp[2]), int(temp[3]), int(temp[4]), int(temp[5]))
                        boxes_lists.append(box)
                        #last
                        label = str(temp[-1])[:-1]
                        label_texts.append(label)
                except:
                    pass

        for i in range(len(boxes_lists)):
            # Crop the images based on bounding boxes (x, y, w, h)
            temp_img = img.copy()
            temp_img = preprocessing(temp_img)
            #region ng box based sa x, y, w, h
            #[y:y+h, x:x+w] - region na ikwaon sa buong frame
            crop_img = temp_img[boxes_lists[i][1]:boxes_lists[i][1]+boxes_lists[i][3], boxes_lists[i][0]:boxes_lists[i][0]+boxes_lists[i][2]]

            if label_texts[i] in helmetLabels:
                cv.imwrite(f"./Helmet/img{data_counter}.jpg", crop_img)
            cv.imwrite(f"./NoHelmet/img{data_counter}.jpg", crop_img)
#Train the images inside Helmet and NoHelmet folders sa Teachable Machine with (epoch 50, batch size 16)
#tried other epochs such as epoch 200 - overfitting happens